# USDe Morpho Benchmarks and Reallocation

In this Jupyter notebook, we will automate the process of analyzing and reallocating USDe liquidity within Morpho's financial pools. 

Our primary focus will lie in assessing the current allocation of USDe, the utilization rates, borrowing rates, optimal rates, and capped rates across different liquidity pools distinguished by their Loan-to-Value (LTV) ratios. We aim to ensure that pool allocations are optimized both for the stable functioning of the overall market and for better returns for liquidity providers and borrowers.

1. **Data Analysis**: We will start by performing a data analysis on the current status of various USDe pools. This will include processing important metrics such as total supply, borrow rates, and optimal rates to identify pools that may require attention or reallocation.

2. **Manual Reallocation Criteria**: Based on the analysis, we'll assess the necessity for manual reallocations in accordance with established criteria for active and inactive pools, including factors like minimum balances, maximum utilization, and the safety margin for withdrawing portions of the allocated USDe supply.

In [68]:
import pandas as pd
import numpy as np
import datetime
import math
import json
from data_manipulation import *
from api import *

pd.set_option('display.max_columns', None)

## Manual Reallocation Analysis

Defined Pools we look at for allocation and necessary values

In [69]:
pool_keys = [
    'USDe 94.50%',
    'USDe 91.50%',
    'USDe 86%',
    'USDe 77%',
    'sUSDe 94.50%',
    'sUSDe 91.50%',
    'sUSDe 86%',
    'sUSDe 77%',
    'PT-sUSDe-24Oct2024 86%'
]
idle_key = [
    'Idle',
]
idle_col = [
    'LLTV',
    'Total Supply', 
    'Maker Allocation',     
    'Utilization', 
    'Borrow Rate',              
]
columns_ordered = [
    'Status', 
    'LLTV',
    'Total Supply', 
    'Maker Allocation', 
    'Utilization', 
    'Borrow Rate',
    'Supply Cap',
    'Total Borrow', 
    'Maker Borrow', 
    'Optimal Rate', 
    'Capped Borrow Rate', 
    'DSR',
    'Fixed Spread',
    'Fixed Spread Rate Value',
    'Fixed Slope per 100M',
    'Fixed Slope',
    'Proportional Spread',
    'Proportional Spread Rate Value',
    'Proportional Spread Fixed Value',
    'Proportional Slope per 100M',
    'Proportional Slope',
    'Proportional Slope Fixed Value',
    'Low Target Threshold',
    'High Target Threshold',
    'Target Borrow Rate',
    'Min Borrow Rate',
    'Max Borrow Rate',
    'Utilization Where Rate Equal to DSR',
    'DSR Adjustment',
    'Total Supply After DSR Adjustment',
    'Maker Supply After DSR Adjustment',
    'Utilization After DSR Adjustment',
    'Inactive Withdrawal',
    'Utilization Where Rate Equal To Min Target',
    'Active Withdrawal',
    'Utilization Where Rate Equal To Max Target',
    'Active Deposits',
    'Manual Adjustment',
    'Total Change',
    'Final Allocation',
    'Final Supply',
    'Final Utilization',
    'Final Borrow Rate',
    'Final Capped Rate',
    'Maker Borrow at Old Utilization',
    'Borrow Rate Change',                   
]

pools_df = pd.DataFrame(index=pool_keys, columns=columns_ordered)
idle_df = pd.DataFrame(index=idle_key, columns=idle_col)
pools_df.head(1)

,Status,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate,Supply Cap,Total Borrow,Maker Borrow,Optimal Rate,Capped Borrow Rate,DSR,Fixed Spread,Fixed Spread Rate Value,Fixed Slope per 100M,Fixed Slope,Proportional Spread,Proportional Spread Rate Value,Proportional Spread Fixed Value,Proportional Slope per 100M,Proportional Slope,Proportional Slope Fixed Value,Low Target Threshold,High Target Threshold,Target Borrow Rate,Min Borrow Rate,Max Borrow Rate,Utilization Where Rate Equal to DSR,DSR Adjustment,Total Supply After DSR Adjustment,Maker Supply After DSR Adjustment,Utilization After DSR Adjustment,Inactive Withdrawal,Utilization Where Rate Equal To Min Target,Active Withdrawal,Utilization Where Rate Equal To Max Target,Active Deposits,Manual Adjustment,Total Change,Final Allocation,Final Supply,Final Utilization,Final Borrow Rate,Final Capped Rate,Maker Borrow at Old Utilization,Borrow Rate Change
USDe 94.50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
idle_df

,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate
Idle,NaN,NaN,NaN,NaN,NaN


### Retrieve latest values

Connect to BA API and retrieve the data for the pools we are interested in. First define pools by their uuid, to retrieve the data.

In [71]:
market_to_pool = {
    "db760246f6859780f6c1b272d47a8f64710777121118e56e0cdb4b8b744a3094": 'USDe 94.50%',
    "8e6aeb10c401de3279ac79b4b2ea15fc94b7d9cfc098d6c2a1ff7b2b26d9d02c": 'USDe 91.50%',
    "c581c5f70bd1afa283eed57d1418c6432cbff1d862f94eaf58fdd4e46afbb67f": 'USDe 86%',
    "fd8493f09eb6203615221378d89f53fcd92ff4f7d62cca87eece9a2fff59e86f": 'USDe 77%',
    "e475337d11be1db07f7c5a156e511f05d1844308e66e17d2ba5da0839d3b34d9": 'sUSDe 94.50%',
    "1247f1c237eceae0602eab1470a5061a6dd8f734ba88c7cdc5d6109fb0026b28": 'sUSDe 91.50%',
    "39d11026eae1c6ec02aa4c0910778664089cdd97c3fd23f68f7cd05e2e95af48": 'sUSDe 86%',
    "42dcfb38bb98767afb6e38ccf90d59d0d3f0aa216beb3a234f12850323d17536": 'sUSDe 77%',
    "8f46cd82c4c44a090c3d72bd7a84baf4e69ee50331d5deae514f86fe062b0748": 'PT-sUSDe-24Oct2024 86%'
}

In [72]:
# market_to_pool.keys()

Retrieve the data from API

In [73]:
# Base URL for market data
base_market_url = "https://morpho-api.blockanalitica.com/markets/{}/historic/?days_ago=0&network=ethereum"
# Base URL for wallet data
base_wallet_url = "https://morpho-api.blockanalitica.com/markets/{}/wallets/?network=ethereum&order=-supply&p_size=4&type=suppliers"
# URL for vault data
vaults_url = "https://morpho-api.blockanalitica.com/vaults/0x73e65dbd630f90604062f6e02fab9138e713edd9/pools/?days_ago=0&network=ethereum&order=-supply_usd&p=1&p_size=15"
# URL for DSR rate
dsr_url = "https://info-sky.blockanalitica.com/save/"
# The wallet address we are interested in
target_wallet = "0x73e65dbd630f90604062f6e02fab9138e713edd9"

# Assuming market_to_pool is defined
api_data = ApiBA(base_market_url, base_wallet_url, market_to_pool.keys(), target_wallet, vaults_url, dsr_url)

With data from API populate the dataframe

In [74]:
market_data = PoolDataHandler(pools_df, market_to_pool)
pools_df = market_data.populate_dataframe(api_data.fetch_data())
pools_df


Updated DataFrame:


,Status,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate,Supply Cap,Total Borrow,Maker Borrow,Optimal Rate,Capped Borrow Rate,DSR,Fixed Spread,Fixed Spread Rate Value,Fixed Slope per 100M,Fixed Slope,Proportional Spread,Proportional Spread Rate Value,Proportional Spread Fixed Value,Proportional Slope per 100M,Proportional Slope,Proportional Slope Fixed Value,Low Target Threshold,High Target Threshold,Target Borrow Rate,Min Borrow Rate,Max Borrow Rate,Utilization Where Rate Equal to DSR,DSR Adjustment,Total Supply After DSR Adjustment,Maker Supply After DSR Adjustment,Utilization After DSR Adjustment,Inactive Withdrawal,Utilization Where Rate Equal To Min Target,Active Withdrawal,Utilization Where Rate Equal To Max Target,Active Deposits,Manual Adjustment,Total Change,Final Allocation,Final Supply,Final Utilization,Final Borrow Rate,Final Capped Rate,Maker Borrow at Old Utilization,Borrow Rate Change
USDe 94.50%,NaN,0.945,204043,155647,0.9302,0.1029,10000000,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDe 91.50%,NaN,0.915,33031571,33031571,0.9225,0.1364,200000000,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDe 86%,NaN,0.86,62658814,62608458,0.9071,0.0939,500000000,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDe 77%,NaN,0.77,1108619,1108619,0.6992,0.0542,1000000000,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 94.50%,NaN,0.945,462726,45051,0.9005,0.1681,10000000,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 91.50%,NaN,0.915,29602980,29251419,0.9879,0.2282,200000000,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 86%,NaN,0.86,76889979,76467229,0.943,0.2165,500000000,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 77%,NaN,0.77,100155,74587,0.9202,0.0238,1000000000,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PT-sUSDe-24Oct2024 86%,NaN,0.86,100899820,100899819,0.9058,0.0995,100000000,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
#index_to_drop = 'sUSDe 77%PT-sUSDe-24Oct2024 86%'
#pools_df = pools_df.drop(index_to_drop)

To see the allocation in the Idle matket:

In [76]:
idle_market_to_pool = {"57f4e42c0707d3ae0ae39c9343dcba78ff79fa663da040eca45717a9b0b0557f": 'Idle'}
idle_api_data = ApiBA(base_market_url, base_wallet_url, idle_market_to_pool.keys(), target_wallet, vaults_url, dsr_url)
idle_data = PoolDataHandler(idle_df, idle_market_to_pool)
idle_df = idle_data.populate_dataframe(idle_api_data.fetch_data())
idle_df


Updated DataFrame:


,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate,Supply Cap,DSR
Idle,0.0,21875119,21875119,0.0,0.0,24519928653854223487948685977518604288,0.06


### Reallocation Metaparameters

They are set as done by Monet. Change only in case you know what you are doing!

In [77]:
realloc_metaparm = {
    'active_pool' : {
        'min_balance' : 10000000,
        'max_utilization' : 0.9,
        'max_portion_to_withdraw' : 0.1,
        'allocation_significance_threshold' : 10000,
    },
    'inactive_pool' : {
        'min_balance' : 100000,
        'max_utilization' : 0.93,
        'max_portion_to_withdraw' : 1,
        'allocation_significance_threshold' : 10000,
    },
}

In [78]:
pools_analysis = PoolAnalysis(pools_df, idle_df, realloc_metaparm)

#### Classify market as Active or Inactive

Change the status of the pool you want to change by changing the pool status to `Active` or `Inactive`

In [79]:
pools_df.loc['USDe 94.50%', 'Status'] = 'Inactive'
pools_df.loc['USDe 91.50%', 'Status'] = 'Active'
pools_df.loc['USDe 86%', 'Status'] = 'Active'
pools_df.loc['USDe 77%', 'Status'] = 'Inactive'
pools_df.loc['sUSDe 94.50%', 'Status'] = 'Inactive'
pools_df.loc['sUSDe 91.50%', 'Status'] = 'Active'
pools_df.loc['sUSDe 86%', 'Status'] = 'Active'
pools_df.loc['sUSDe 77%', 'Status'] = 'Inactive'
pools_df.loc['PT-sUSDe-24Oct2024 86%', 'Status'] = 'Active'

## Vault benchmarks

Here you can modify the vault benchmark calculations.

In [80]:
# Tweakable parameters
pools_df['Fixed Spread'] = 3 / 100
pools_df['Fixed Slope per 100M'] = 0.75 / 100

pools_df['Proportional Spread'] = 65 / 100
pools_df['Proportional Slope per 100M'] = 5 / 100

pools_df['Low Target Threshold'] = 80 / 100
pools_df['High Target Threshold'] = 110 / 100

# Indirect parameters
pools_df['Fixed Spread Rate Value'] = pools_df['DSR'] + pools_df['Fixed Spread']
pools_df['Fixed Slope'] = pools_df['Fixed Slope per 100M'] / 100000000
pools_df['Proportional Spread Rate Value'] = pools_df['DSR'] * (1 + pools_df['Proportional Spread'])
pools_df['Proportional Spread Fixed Value'] = pools_df['Proportional Spread Rate Value'] - pools_df['DSR']
pools_df['Proportional Slope'] = pools_df['Proportional Slope per 100M'] / 100000000
pools_df['Proportional Slope Fixed Value'] = pools_df['Proportional Spread Rate Value'] * pools_df['Proportional Slope per 100M']

In [81]:
#pools_analysis.define_active_or_inactive()

### Manual Adjustment

If you want to add a manual adjustment, do it here by uncommenting the line of code with pool you want to adjust and change the number inside the parenthesis.

In [82]:
#pools_df.loc['USDe 94.50%', 'Manual Adjustment'] = int(0)
#pools_df.loc['USDe 91.50%', 'Manual Adjustment'] = int(0)
#pools_df.loc['USDe 86%', 'Manual Adjustment'] = int(0)
#pools_df.loc['USDe 77%', 'Manual Adjustment'] = int(0)
#pools_df.loc['sUSDe 94.50%', 'Manual Adjustment'] = int(0)
#pools_df.loc['sUSDe 91.50%', 'Manual Adjustment'] = int(0)
#pools_df.loc['sUSDe 86%', 'Manual Adjustment'] = int(0)
#pools_df.loc['sUSDe 77%', 'Manual Adjustment'] = int(0)
#pools_df.loc['PT-sUSDe-24Oct2024 86%', 'Manual Adjustment'] = int(0)

### Reallocation Dataframe

Get the final table with all data. For easier overview focus on the following tables

In [83]:
pools_analysis = PoolAnalysis(pools_df, idle_df, realloc_metaparm)

In [84]:
pools_analysis.update_pool_dataframe()

,Status,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate,Supply Cap,Total Borrow,Maker Borrow,Optimal Rate,Capped Borrow Rate,DSR,Fixed Spread,Fixed Spread Rate Value,Fixed Slope per 100M,Fixed Slope,Proportional Spread,Proportional Spread Rate Value,Proportional Spread Fixed Value,Proportional Slope per 100M,Proportional Slope,Proportional Slope Fixed Value,Low Target Threshold,High Target Threshold,Target Borrow Rate,Min Borrow Rate,Max Borrow Rate,Utilization Where Rate Equal to DSR,DSR Adjustment,Total Supply After DSR Adjustment,Maker Supply After DSR Adjustment,Utilization After DSR Adjustment,Inactive Withdrawal,Utilization Where Rate Equal To Min Target,Active Withdrawal,Utilization Where Rate Equal To Max Target,Active Deposits,Manual Adjustment,Total Change,Final Allocation,Final Supply,Final Utilization,Final Borrow Rate,Final Capped Rate,Maker Borrow at Old Utilization,Borrow Rate Change
USDe 94.50%,Inactive,0.945,204043,155647,0.9302,0.1029,10000000,189800,144782,0.054,0.054,0.06,0.03,0.09,0.0075,7.500000e-11,0.65,0.099,0.039,0.05,5.000000e-10,0.00495,0.8,1.1,0.1151,0.0921,0.1266,0.9037,0,204043,155647,0.9302,0,0.9235,0,0.9448,0,0,0,155647,204043,0.9302,0.1029,0.054,144782,0.0
USDe 91.50%,Active,0.915,33031571,33031571,0.9225,0.1364,200000000,30471624,30471624,0.0814,0.0814,0.06,0.03,0.09,0.0075,7.500000e-11,0.65,0.099,0.039,0.05,5.000000e-10,0.00495,0.8,1.1,0.1151,0.0921,0.1266,0.5845,0,33031571,33031571,0.9225,0,0.9044,0,0.9185,143849.794774,0,143849.794774,33175420,33175420,0.9185,0.1266,0.0814,30604324,-0.0098
USDe 86%,Active,0.86,62658814,62608458,0.9071,0.0939,500000000,56837810,56792132,0.0774,0.0774,0.06,0.03,0.09,0.0075,7.500000e-11,0.65,0.099,0.039,0.05,5.000000e-10,0.00495,0.8,1.1,0.1151,0.0921,0.1266,0.6302,0,62658814,62608458,0.9071,0,0.9063,0,0.9212,0,0,0,62608458,62658814,0.9071,0.0939,0.0774,56792132,0.0
USDe 77%,Inactive,0.77,1108619,1108619,0.6992,0.0542,1000000000,775146,775146,0.0651,0.0542,0.06,0.03,0.09,0.0075,7.500000e-11,0.65,0.099,0.039,0.05,5.000000e-10,0.00495,0.8,1.1,0.1151,0.0921,0.1266,0.806,-146899,961720,961720,0.806,-128229,0.9138,0,0.9315,0,0,-275128,833491,833491,0.93,0.1237,0.0651,582776,0.0695
sUSDe 94.50%,Inactive,0.945,462726,45051,0.9005,0.1681,10000000,416684,40568,0.1656,0.1656,0.06,0.03,0.09,0.0075,7.500000e-11,0.65,0.099,0.039,0.05,5.000000e-10,0.00495,0.8,1.1,0.1151,0.0921,0.1266,0.1348,0,462726,45051,0.9005,-14678,0.3674,0,0.6174,0,0,-14678,30373,448048,0.93,0.3146,0.1656,27350,0.1465
sUSDe 91.50%,Active,0.915,29602980,29251419,0.9879,0.2282,200000000,29244783,28897475,0.0627,0.0627,0.06,0.03,0.09,0.0075,7.500000e-11,0.65,0.099,0.039,0.05,5.000000e-10,0.00495,0.8,1.1,0.1151,0.0921,0.1266,0.8483,0,29602980,29251419,0.9879,0,0.9156,0,0.934,1708350.835118,0,1708350.835118,30959769,31311330,0.934,0.1267,0.0627,30585155,-0.1015
sUSDe 86%,Active,0.86,76889979,76467229,0.943,0.2165,500000000,72507250,72108596,0.0945,0.0945,0.06,0.03,0.09,0.0075,7.500000e-11,0.65,0.099,0.039,0.05,5.000000e-10,0.00495,0.8,1.1,0.1151,0.0921,0.1266,0.4619,0,76889979,76467229,0.943,0,0.8695,0,0.9113,2674653.941951,0,2674653.941951,79141882,79564632,0.9113,0.1265,0.0945,74630794,-0.09
sUSDe 77%,Inactive,0.77,100155,74587,0.9202,0.0238,1000000000,92162,68634,0.0148,0.0148,0.06,0.03,0.09,0.0075,7.500000e-11,0.65,0.099,0.039,0.05,5.000000e-10,0.00495,0.8,1.1,0.1151,0.0921,0.1266,1,-155,100000,74432,0.9216,0,1,0,1,0,0,0,74587,100155,0.9202,0.0238,0.0148,68634,0.0
PT-sUSDe-24Oct2024 86%,Active,0.86,100899820,100899819,0.9058,0.0995,100000000,91395056,91395055,0.0848,0.0848,0.06,0.03,0.09,0.0075,7.500000e-11,0.65,0.099,0.039,0.05,5.000000e-10,0.00495,0.8,1.1,0.1151,0.0921,0.1266,0.5491,0,100899820,100899819,0.9058,0,0.9029,0,0.9164,0,0,0,100899819,100899820,0.9058,0.0996,0.0848,91395056,0.0001


Save the data if you want to perform later analysis or visualizations on this

In [85]:
#pools_analysis.pool_df.to_json(f'reallocations_{datetime.datetime.now().strftime('%Y_%m_%d')}.json')

### Average Utilization, Net and Running Change

Some basic statistics for reallocation

In [86]:
average_utilization = pools_analysis.pool_df['Utilization'].mean()
print(f'The average utilization of the pools is: {average_utilization * 100:.2f}%')

The average utilization of the pools is: 90.18%


The net changes based on the type of adjusment are as follows

In [87]:
net_change_df = pools_analysis.pool_df[['DSR Adjustment', 'Inactive Withdrawal', 'Active Withdrawal', 'Active Deposits', 'Manual Adjustment']].agg('sum').to_frame().T
net_change_df.columns = ['DSR Adjustment Sum', 'Inactive Withdrawal Sum', 'Active Withdrawal Sum', 'Active Deposits Sum', 'Manual Adjustment Sum']
net_change_df = net_change_df.round(0).astype(int)

net_change_df

,DSR Adjustment Sum,Inactive Withdrawal Sum,Active Withdrawal Sum,Active Deposits Sum,Manual Adjustment Sum
0,-147054,-142907,0,4526854,0


The running total of changes by the type of adjustment are as follows, which will result in new adjusted Idle supply

In [88]:
cumulative_sums = net_change_df.cumsum(axis=1).iloc[-1]
cumulative_sums_df = pd.DataFrame({
    'DSR Adjustment Sum': [cumulative_sums['DSR Adjustment Sum']],
    'Inactive Withdrawal Sum': [cumulative_sums['Inactive Withdrawal Sum']],
    'Active Withdrawal Sum': [cumulative_sums['Active Withdrawal Sum']],
    'Active Deposits Sum': [cumulative_sums['Active Deposits Sum']],
    'Manual Adjustment Sum': [cumulative_sums['Manual Adjustment Sum']]
})
cumulative_sums_df = cumulative_sums_df.round(0).astype(int)

cumulative_sums_df

,DSR Adjustment Sum,Inactive Withdrawal Sum,Active Withdrawal Sum,Active Deposits Sum,Manual Adjustment Sum
0,-147054,-289961,-289961,4236893,4236893


## Manual Reallocation Dataframe

The following table shows the change and new allocated supply with wei allocation

In [89]:
new_pool_data = {
    'Net Change': pools_df['Total Change'],
    'New Allocation Supply': pools_df['Final Allocation']
}
manual_realloaction_pools = pd.DataFrame(new_pool_data)
manual_realloaction_pools['New Allocation Wei'] = manual_realloaction_pools['New Allocation Supply'] * pow(10, 18)

In [90]:
# Create a styled DataFrame
styled_df = manual_realloaction_pools.style.format({
    'Net Change': '{:.0f}',
    'New Allocation Supply': '{:.0f}',
    'New Allocation Wei': '{:.0f}'
}).map(PoolAnalysis.color_net_change, subset=['Net Change'])

styled_df

,Net Change,New Allocation Supply,New Allocation Wei
USDe 94.50%,0,155647,155646999999999989776384
USDe 91.50%,143850,33175420,33175420000000001871708160
USDe 86%,0,62608458,62608458000000003440902144
USDe 77%,-275128,833491,833490999999999942590464
sUSDe 94.50%,-14678,30373,30373000000000000786432
sUSDe 91.50%,1708351,30959769,30959769000000001429995520
sUSDe 86%,2674654,79141882,79141882000000004233625600
sUSDe 77%,0,74587,74586999999999999213568
PT-sUSDe-24Oct2024 86%,0,100899819,100899818999999999114215424


In [91]:
print(f'The total net change in allocation is: ${manual_realloaction_pools["Net Change"].sum():.0f}')

The total net change in allocation is: $4237049


## Pool Overview Dataframe

A brief statistics for pool overview

In [92]:
pool_stats = [
    'Total Non-Idle Allocation',
    'Supply Weighted LLTV',
    'Supply Weighted sUSDe',
    'Average Borrow Rate',
    'Average Capped Rate',
    'Rate at Prior Equilibrium',
]
columns_stats = [
    'Current',
    'Future',
    'Change'                 
]

pool_overview = pd.DataFrame(index=pool_stats, columns=columns_stats)
statistics = PoolOverview(pools_df, pool_overview)

In [93]:
statistics.update_pool_overview()

,Current,Future,Change
Total Non-Idle Allocation,303642400,307879446,0.013954
Supply Weighted LLTV,0.870987,0.871243,0.000294
Supply Weighted sUSDe,0.348562,0.357954,0.026944
Average Borrow Rate,0.145547,0.111159,-0.236268
Average Capped Rate,0.083045,0.083075,0.000362
Rate at Prior Equilibrium,0.145547,0.111159,-0.236268


In [94]:
pool_styled_df = pool_overview.style.format({
    'Current': '{:.3f}',
    'Future': '{:.3f}',
    'Change': '{:.2f}%'
}).map(PoolAnalysis.color_net_change, subset=['Change'])

pool_styled_df

,Current,Future,Change
Total Non-Idle Allocation,303642400.000,307879446.000,0.01%
Supply Weighted LLTV,0.871,0.871,0.00%
Supply Weighted sUSDe,0.349,0.358,0.03%
Average Borrow Rate,0.146,0.111,-0.24%
Average Capped Rate,0.083,0.083,0.00%
Rate at Prior Equilibrium,0.146,0.111,-0.24%


### Current Debt Ceiling

In [95]:
current_ddm_target_debt = int(pool_overview.at['Total Non-Idle Allocation', 'Current'] + idle_df.at['Idle', 'Total Supply'])

print(f'The current DDM target debt is: ${current_ddm_target_debt}')

future_ddm_target_debt = math.ceil((int(pool_overview.at['Total Non-Idle Allocation', 'Future'])+ 5000000)/ 25000000) * 25000000

print(f'The future DDM target debt is: ${future_ddm_target_debt}')

The current DDM target debt is: $325517519
The future DDM target debt is: $325000000


## JSON Logic

Include only pools that change in allocation

### Allocation to USDe

[
MakerDAO: Dai Token (DAI),
Ethena: USDe (USDe),
Morpho: MorphoChainlinkOracleV2,
Morpho: AdaptiveCurveIRM,
LLTV (times 10E16)
],
Allocation WEI

### Allocation to sUSDe

[
MakerDAO: Dai Token (DAI),
Ethena: Staked USDe (sUSDe),
Morpho: MorphoChainlinkOracleV2,
Morpho: AdaptiveCurveIRM,
LLTV (times 10E16)
]
Allocation WEI

### Everything else goes to the idle market

[
MakerDAO: Dai Token (DAI),
Burn Address,
Burn Address,
Burn Address,
0
]
Uint256 Maximum Value

In [96]:
manual_reallocation_pools = manual_realloaction_pools

manual_reallocation_pools.index = [
    'USDe 94.50%', 'USDe 91.50%', 'USDe 86%', 'USDe 77%', 
    'sUSDe 94.50%', 'sUSDe 91.50%', 'sUSDe 86%', 'sUSDe 77%', 
    'PT-sUSDe-24Oct2024 86%'
]

# Define the base address values - Source: https://docs.morpho.org/addresses/
base_addresses = {
    "USDe": [
        "0x6B175474E89094C44Da98b954EedeAC495271d0F", # DAI
        "0x4c9EDD5852cd905f086C759E8383e09bff1E68B3", # USDe
        "0xaE4750d0813B5E37A51f7629beedd72AF1f9cA35", # MorphoChainlinkOracleV2
        "0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC" # AdaptiveCurveIRM
    ],
    "sUSDe": [
        "0x6B175474E89094C44Da98b954EedeAC495271d0F", # DAI
        "0x9D39A5DE30e57443BfF2A8307A4256c8797A3497", # sUSDe
        "0x5D916980D5Ae1737a8330Bf24dF812b2911Aae25", # MorphoChainlinkOracleV2
        "0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC" # AdaptiveCurveIRM
    ],
    "PT-sUSDe-24Oct2024": [
        "0x6B175474E89094C44Da98b954EedeAC495271d0F", # DAI
        "0xAE5099C39f023C91d3dd55244CAFB36225B0850E", # PT-sUSDe-24Oct2024
        "0xaE4750d0813B5E37A51f7629beedd72AF1f9cA35", # MorphoChainlinkOracleV2
        "0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC" # AdaptiveCurveIRM
    ],
    "Idle": [
        "0x6B175474E89094C44Da98b954EedeAC495271d0F", # DAI
        "0x0000000000000000000000000000000000000000", # Idle
        "0x0000000000000000000000000000000000000000", # MorphoChainlinkOracleV2
        "0x0000000000000000000000000000000000000000" # AdaptiveCurveIRM
    ]
}

# Function to determine the pool type based on the allocation type (in the index)
def determine_pool_type(allocation_type):
    if "USDe" in allocation_type:
        return base_addresses["USDe"]
    elif "sUSDe" in allocation_type:
        return base_addresses["sUSDe"]
    elif "PT-sUSDe-24Oct2024" in allocation_type:
        return base_addresses["PT-sUSDe-24Oct2024"]
    else:
        return base_addresses["Idle"]

# Initialize the JSON structure
output_json = []

for allocation_type, row in manual_reallocation_pools.iterrows():
    if row['Net Change'] != 0:
        pool_type = determine_pool_type(allocation_type)
        lltv = str(int(float(allocation_type.split()[1].replace('%', '')) * 10**16))  # Convert percentage to LLTV
        pool = pool_type + [lltv]

        allocation_wei = str(row['New Allocation Wei'])  # Convert to string
        
        output_json.append([pool, allocation_wei])

# Add the 'Idle' pool entry
idle_pool = base_addresses["Idle"] + ["0"]
idle_allocation_wei = "0xffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff"
output_json.append([idle_pool, idle_allocation_wei])

# Convert the list to JSON formatted string
output_str = json.dumps(output_json, indent=2)

# Print or save the JSON formatted string
print(output_str)

# Optionally, save to a file
with open("morpho_vault_realloc.json", "w") as file:
    file.write(output_str)

[
  [
    [
      "0x6B175474E89094C44Da98b954EedeAC495271d0F",
      "0x4c9EDD5852cd905f086C759E8383e09bff1E68B3",
      "0xaE4750d0813B5E37A51f7629beedd72AF1f9cA35",
      "0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC",
      "915000000000000000"
    ],
    "33175420000000000000000000"
  ],
  [
    [
      "0x6B175474E89094C44Da98b954EedeAC495271d0F",
      "0x4c9EDD5852cd905f086C759E8383e09bff1E68B3",
      "0xaE4750d0813B5E37A51f7629beedd72AF1f9cA35",
      "0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC",
      "770000000000000000"
    ],
    "833491000000000000000000"
  ],
  [
    [
      "0x6B175474E89094C44Da98b954EedeAC495271d0F",
      "0x4c9EDD5852cd905f086C759E8383e09bff1E68B3",
      "0xaE4750d0813B5E37A51f7629beedd72AF1f9cA35",
      "0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC",
      "945000000000000000"
    ],
    "30373000000000000000000"
  ],
  [
    [
      "0x6B175474E89094C44Da98b954EedeAC495271d0F",
      "0x4c9EDD5852cd905f086C759E8383e09bff1E68B3",
      "0xaE4750d081